In [34]:
# Import libraries
from __future__ import annotations

import json 
import sympy
from matplotlib import pyplot as plt

In [35]:
# CRN Class

from sympy import Symbol

class Crn:
    def __init__(self, equation: str, initial_condition=None):
        self.equation = equation
        self.parts = self.split()
        self.lhs = self.parts[0]
        self.dv = self.parts[0]
        self.rhs = self.parts[1]
        self.expr = sympy.parse_expr(self.parts[1])
        self.expr_parts = self.split_rhs()
        self.pos_terms = self.expr_parts[0]
        self.neg_terms = self.expr_parts[1]
        self.p_poly = self.gen_p_poly()
        self.pos = self.rewrite_pos()
        self.neg = self.rewrite_neg()
        self.system = self.write_system()

    def split(self):  
        parts = self.equation.split("=")
        return parts

    def gen_p_poly(self):
        sym = self.dv[:-1]
        sym_1 = Symbol(f"{sym}_1")
        sym_2 = Symbol(f"{sym}_2")
        p_plus = Symbol("p^{+}")
        p_minus = Symbol("p^{-}")
        poly = (p_plus+p_minus)*sym_1*sym_2
        return poly

    def split_rhs(self):
        ispos = lambda x: x.as_coeff_Mul()[0].is_positive
        pos, neg = sympy.sift(sympy.Add.make_args(self.expr), ispos, binary=True)
        return pos, neg
    
    def rewrite_pos(self):
        pos_expr = ""
        for term in self.pos_terms:
            pos_expr += str(term) + "+" 
        pos_expr = pos_expr[:-1]
        return sympy.parse_expr(pos_expr)

    def rewrite_neg(self):
        neg_expr = ""
        for term in self.neg_terms:
            neg_expr += str(term)
        neg_expr = neg_expr[1:]
        return neg_expr

    def write_system(self):
        first_dv = Symbol(f"{self.dv}_1")
        second_dv = Symbol(f"{self.dv}_2")
        return f"{first_dv}={self.pos}-{self.p_poly}", f"{second_dv}={self.neg}-{self.p_poly}"

    def as_latex(self):
        return sympy.latex(str(self))
        

    def __str__(self):
        return f"""
        -------------------------------------------------------
        CrnWriter
        -------------------------------------------------------
        [Equation]
        Equation: {self.equation}
        Dependent variable (LHS): {self.dv}
        RHS: {self.rhs}
        -------------------------------------------------------
        [RHS Parse]
        Expression: {self.expr}
        Positive Terms: {self.pos_terms} 
        Negative Terms: {self.neg_terms}
        Positive Expression: {self.pos}
        Negative Expression {self.neg}
        -------------------------------------------------------
        [P-Term]
        P Polynomial: {self.p_poly}
        -------------------------------------------------------
        [Rewritten]
        System: 
            {self.system[0]}
            {self.system[1]}
        """


In [44]:
class CrnConfig:
    def __init__(self, filepath: str) -> CrnConfig:
        with open(filepath) as f:
            self.cfg = json.load(f)
        self.equations = []
        self.valid = self.process_config()
        if not self.valid:
            print("Invalid configuration.")
        print(self.parse_to_equations())


    def process_config(self) -> bool:
        system = self.cfg.get("system")
        if not system:
            self.system = None
            return False
        equations = system.get("equations")
        if not equations:
            self.equations = []
        self.equations = equations 
        init_condition = system.get("init")
        if not init_condition:
            self.init = None
            return False
        self.init = init_condition
        params = system.get("params")
        if not params:
            self.params = None
            return False
        self.params = params
        return True

    def parse_to_equations(self):
        for equation in self.equations:
            try:
                lhs = equation["lhs"]
                rhs = equation["rhs"]
                crn = Crn(equation=f"{lhs}={rhs}")
                print(crn)
            except Exception as e:
                print(e)
    
        
            

In [45]:
x = CrnConfig(filepath="/Users/anishsinha/Home/personal/research/crn/system.json")


        -------------------------------------------------------
        CrnWriter
        -------------------------------------------------------
        [Equation]
        Equation: x=sigma * (y-x)
        Dependent variable (LHS): x
        RHS: sigma * (y-x)
        -------------------------------------------------------
        [RHS Parse]
        Expression: sigma*(-x + y)
        Positive Terms: [sigma*(-x + y)] 
        Negative Terms: []
        Positive Expression: sigma*(-x + y)
        Negative Expression 
        -------------------------------------------------------
        [P-Term]
        P Polynomial: _1*_2*(p^{+} + p^{-})
        -------------------------------------------------------
        [Rewritten]
        System: 
            x_1=sigma*(-x + y)-_1*_2*(p^{+} + p^{-})
            x_2=-_1*_2*(p^{+} + p^{-})
        

        -------------------------------------------------------
        CrnWriter
        -------------------------------------------------------
  